In [1]:
import os
import concurrent.futures
import multiprocess as mp
import random


from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
import lightkurve as lk
from astroquery.simbad import Simbad
import pandas as pd 
from numpy import genfromtxt


#tf
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
hdul = fits.open('/Users/michaelsong/Documents/GitHub/starzam/Labels/J_ApJS_236_42_table2.dat.gz.fits')

hdul.info()
print(hdul[1].data)

Filename: /Users/michaelsong/Documents/GitHub/starzam/Labels/J_ApJS_236_42_table2.dat.gz.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     114   ()      
  1  table2.dat    1 TableHDU       162   16094R x 20C   [I8, I4, I3, F5.3, F5.3, F5.2, F4.2, F4.2, F4.2, F5.2, F4.2, F4.2, F4.2, F5.2, F4.2, F4.2, F4.2, F5.2, F5.2, I1]   
[(757137, 4751, 139, 2.383, 0.012, -0.08, 0.3, 1.7, 0.14, 13.91, 0.41, 1.55, 0.13, 13.26, 0.39, 1.72, 0.14, 13.97, 0.41, 1)
 (892010, 4834, 151, 2.161, 0.023, 0.21, 0.3, 1.42, 0.29, 16.37, 1.38, 1.36, 0.28, 16.02, 1.34, 1.43, 0.3, 16.47, 1.39, 0)
 (892738, 4534, 135, 1.769, 0.022, -0.25, 0.3, 1.15, 0.21, 23.15, 1.62, 0.99, 0.18, 21.49, 1.48, 1.12, 0.2, 22.86, 1.6, 0)
 ...
 (12885196, 4846, 142, 2.878, 0.008, 0.07, 0.3, 1.55, 0.1, 7.5, 0.17, 1.46, 0.09, 7.29, 0.16, 1.57, 0.1, 7.55, 0.17, 1)
 (12885373, 4855, 144, 2.627, 0.009, 0.12, 0.3, 1.49, 0.09, 9.83, 0.22, 1.42, 0.09, 9.59, 0.22, 1.51, 0.1, 9.9, 0.22, 1)
 (12

In [3]:
test_lk = lk.search_lightcurve('KIC '+str(757137), author='Kepler',exptime=1800)[2].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
print(len(test_lk.time.value))
l = random.randrange(250, len(test_lk.time.value))
s = random.randrange(0,len(test_lk.time.value)-l)
print(s,l)
test_lk2 = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
test_periodogram = test_lk2.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
freq = np.asarray([test_periodogram.frequency,test_periodogram.power*100000])
print(freq[1])

4134
2884 1146
[12.64322393  3.01461823  5.1850582   3.78007941  4.72722984  6.61668392
  4.61931635  3.08647375  6.78547504  3.17914494  4.80879995  0.9935621
  4.39004802 10.14670651  8.6282496   6.45913251 12.17769887  5.34243587
  3.54330461 16.17302037  5.71573001  6.05717721  5.10344678  4.37293807
  1.27238123 10.49138001  6.81652618  2.45898988  0.5161188   4.28121559
  1.43389115  0.74330265  2.64172001  2.76112893  1.27997778  0.33767567
  2.97810698  1.41028003  1.19041698  1.89767808  1.59331636  1.26902919
  1.15664369  2.59023616  0.87124678  1.44719482  1.32514662  1.18635239
  1.3480541   0.36700124  0.11968683  0.78818292  0.89265212  0.4154108
  1.02452335  1.12221132  0.20735838  0.42419001  0.46519454  1.36561205
  1.24395541  0.4567286   0.60627518  0.83884407  0.48369706  0.54817106
  0.50165938  0.68842418  0.589237    0.98896752  0.43887154  0.72773679
  0.9261075   0.52177779  0.84000301  0.38464013  0.28739114  0.66712479
  0.8148812   0.25357022  0.83808964  

In [4]:
def prepsr(i,sr,j):
    r = True
    if r:
        test_lk = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
        l = random.randrange(240, len(test_lk.time.value))
        s = random.randrange(0,len(test_lk.time.value)-l)
        lightcurve = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
    else:
        lightcurve = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
    
    periodogram = lightcurve.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
    # periodogram = periodogram.bin(binsize=10, method='mean')
    freq = np.asarray([periodogram.frequency,periodogram.power*100000])
    # pd.DataFrame(freq[1]).to_csv("/content/drive/MyDrive/Research - Yihong Song and Rohit Prasanna/Yihong/Data/"+str(labeled_data["Kepler_ID"][i])+"_"+str(j)+".csv")
    return np.append(np.append(np.append(np.append(hdul[1].data[i][0:2],hdul[1].data[i][3]),hdul[1].data[i][5]),[j]),freq[1])

def getlc(i):
    lc_data = []
    sr = lk.search_lightcurve('KIC '+str(hdul[1].data[i][0]), author='Kepler',exptime=1800)
    # return prepsr(i,sr,1)
    for j in range(len(sr)):
        # x = wrap(prepsr(i,sr,j))
        # if x is not None:
        lc_data.append(prepsr(i,sr,j))
        # y = wrap(external_function_raises)
        # if y is not None:
        #     lc_data.append(y)
    return lc_data

def getdata(start):
    # for k in range(start,int(191449/100)+1,1):
    for k in range(start,start+1,1):
        lc_data = []
        for i in range(k*10,(k+1)*10,1):
            try:
                lc_data = lc_data+getlc(i)
            except:
                continue
            print("\r"+str(i),end="")
        # print(lc_data)
        pd.DataFrame(lc_data).to_csv("/Users/michaelsong/Documents/GitHub/starzam/Data_v5/"+str(1610+k)+".csv")

In [16]:
getdata(0)

9

In [11]:
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     print(executor.map(getdata, [1,2,3]))

with mp.Pool(8) as pool:
    print(pool.map(getdata, [x for x in range(1600,1610)]))

16065

16089[None, None, None, None, None, None, None, None, None, None]
